# Install PyRadios

In [1]:
! pip install pyradios
! pip install geopy

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Import PyRadios & Create instance of RadioBrowser

In [2]:
from pyradios import RadioBrowser
from geopy.geocoders import Nominatim
from random import choice
from IPython.display import FileLink

rb = RadioBrowser()
geolocator = Nominatim(user_agent="TunerTerrace/1.0")

# Find Country/State/City associated with longitude and latitude

We'll try to find a station in Kathmandu:
27.7172° N, 85.3240° E

Note that geopy accepts all kinds of longitude/latitude formats. N/S  vs. positive/negative, include or don't include degrees are all fine.

In [29]:
def reverse_geo(long_lat, geolocator):

    location = geolocator.reverse(long_lat, language='en') 
    country = location.raw['address']['country']
    state = location.raw['address']['state']
    if state == 'Autonomous City of Buenos Aires':
        state = 'Buenos Aires' #I don't have a a better way to deal with this haha
    try: 
        state_district = location.raw['address']['state_district']
    except KeyError:
        state_district = None
    try:
        city = location.raw['address']['city']
    except KeyError:
        city = None
    return [country, state, state_district, city]

In [4]:
reverse_geo('27.7172° N, 85.3240° E', geolocator)

['Nepal', 'Central Development Region', 'Bagmati', 'Kathmandu']

# Find radio stations in/near that city

In [136]:
def get_stations(loc_ls, rb):
    country = loc_ls[0] 
    state = loc_ls[1] 
    state_district = loc_ls[2] 
    if loc_ls[3] == 'Autonomous City of Buenos Aires': # Super specific case I don't know how else to deal with
        city = 'Buenos Aires'
    else:
        city =loc_ls[3]
    # Find all stations in country
    sta_country = rb.stations_bycountry(country=country)
    print("Number of Stations in Country: " + str(len(sta_country)))
    # If only one then you're done
    if len(sta_country) == 1:
        choice_ls = sta_country
    # If none then error
    elif len(sta_country)==0:
        raise('No Station Found')
    
    # Otherwise find all stations in state or state district
    else:
        sta_state = rb.stations_bystate(state=state) 
        if state_district:
            sta_state_dist = rb.stations_bystate(state=state_district) 
        else:
            sta_state_dist = []
        sta_state = sta_state + sta_state_dist
        
        # If more stations in the state then the country than something went wrong, return empty list
        if len(sta_state) > len(sta_country) or len(sta_state)==0:
            sta_state == []
            choice_ls = sta_country
        else:
            print("Number of Stations in State/Province/etc: " + (str(len(sta_state))))
            # If only one station in the state then you're done
            if len(sta_state) == 1:
                choice_ls = sta_state
            # Otherwise find all stations in the city
            else:
                sta_city = rb.stations_bytag(tag=city) 
                if len(sta_city) > len(sta_state) or len(sta_city)==0:
                    sta_city = []
                    print("Number of Stations in City: " + (str(len(sta_city))))
                    choice_ls = sta_state
                else:
                    print("Number of Stations in City: " + (str(len(sta_city))))
                    choice_ls = sta_city
            
            
    return choice_ls

# Pick a station at random from the list

In [137]:
def pick_stations(choice_ls):
    play_me = choice(choice_ls)
    print(play_me['name'] + ' @:')

    print(play_me['url'])
    return play_me

# Put it all together...

In [138]:
loc_ls = reverse_geo('27.7172° N, 85.3240° E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 7
Number of Stations in State/Province/etc: 3
Number of Stations in City: 0
Capital FM 92.4 @:
http://streaming.softnep.net:8037/;


There are no stations tagged as being from Kathmandu specifically, but there are 3 stations in the State District that Kathmandu is in, Bagmati.  Therefore, we randomly select a station from that list of 3 stations.

# Try Adelaide, Australia

34.9285° S, 138.6007° E

In [139]:
loc_ls = reverse_geo('34.9285° S, 138.6007° E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 261
Number of Stations in State/Province/etc: 22
Number of Stations in City: 4
ABC Local Radio 891 Adelaide (MP3) @:
http://live-radio01.mediahubaustralia.com/5LRW/mp3/


There are 4 stations tagged as being from Adelaide, so randomly pick one.

# Try Tonga!
21.1790° S, 175.1982° W

In [140]:
loc_ls = reverse_geo('21.1790° S, 175.1982° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 1
Radio Tonga 101.7 Nuku'alofa @:
http://tbcletio.serverroom.us:8142/;stream.mp3


This time there is 1 station in the whole country, so just return that one.

# Try Buenos Aires!

34.6037° S, 58.3816° W

In [141]:
loc_ls = reverse_geo('34.6037° S, 58.3816° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 284
Number of Stations in State/Province/etc: 106
Number of Stations in City: 13
FM Del 15 - 96.1 Mhz @:
http://server.laradio.online:8419/;?1561750624655


# Try LA!

In [142]:
loc_ls = reverse_geo('34.0522° N, 118.2437° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)


Number of Stations in Country: 3674
Number of Stations in State/Province/etc: 438
Number of Stations in City: 43
UCLA Radio.com - Los Angeles, CA @:
http://uclaradio.com:8000/;


This isn't totally perfect, because it returns a lot of stations for this densely populated area, and one of them is technically in compton, but it's still pretty good.

# Try Ghana!

7.9465° N, 1.0232° W

In [143]:
loc_ls = reverse_geo('7.9465° N, 1.0232° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 10
Oman FM @:
http://stream.zenolive.com/g0cuyd4h5xmtv


# Try Tunis! 
33.8869° N, 9.5375° E

In [144]:
loc_ls = reverse_geo('17.3616° N, 82.5459° E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 244
Number of Stations in State/Province/etc: 3
Number of Stations in City: 0
Radio tapaka Telugu @:
https://mirchitapakahd-lh.akamaihd.net/i/mirchitapakahdlive_1_1@322572/master.m3u8


# Try Seattle!

In [145]:
loc_ls = reverse_geo('47.6062° N, 122.3321° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 3720
Number of Stations in State/Province/etc: 138
Number of Stations in City: 10
KEXP 90.3 Seattle, WA @:
http://live-mp3-128.kexp.org:8000


# Try Tokyo!

In [ ]:
loc_ls = reverse_geo('47.6062° N, 122.3321° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)